In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
!pip install --upgrade accelerate
!pip install -y transformers accelerate
!pip install transformers accelerate



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [2]:
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install datasets

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [5]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import evaluate # Used in place of load_metric from datasets. works in similar manner but load_metric is not available in latest version of datasets.
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
#!wget https://raw.githubusercontent.com/atharvgalphade/Text-Summarizer-Project/main/summarizer-data.zip

!unzip summarizer-data.zip

Archive:  summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [9]:
dataset_samsum=load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
split_lengths=[len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][1]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [11]:
def convert_examples_to_features(example_batch):
    # Tokenizes and converts a batch of examples (dialogues and summaries) into model-friendly features.

    # Tokenize the 'dialogue' field of the example batch.
    # 'max_length=1024' ensures the input doesn't exceed 1024 tokens, and 'truncation=True' truncates longer inputs.
    input_encodings = tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True
    )

    # Temporarily switch the tokenizer to "target tokenizer" mode to process the target text (summary).
    # This is required for some tokenizers, like T5, where input and output might have different tokenization rules.
    with tokenizer.as_target_tokenizer():
        # Tokenize the 'summary' field of the example batch.
        # 'max_length=128' ensures the summary doesn't exceed 128 tokens, and 'truncation=True' truncates longer summaries.
        target_encodings = tokenizer(
            example_batch['summary'],
            max_length=128,
            truncation=True
        )

    # Return a dictionary containing tokenized inputs and targets.
    # 'input_ids' are the tokenized IDs of the input (dialogue),
    # 'attention_mask' indicates which tokens are padding (1 for real tokens, 0 for padding),
    # 'labels' are the tokenized IDs of the target (summary).
    return {
        'input_ids': input_encodings['input_ids'],         # Token IDs for the dialogue (input).
        'attention_mask': input_encodings['attention_mask'], # Attention mask for the input.
        'labels': target_encodings['input_ids']           # Token IDs for the summary (output/target).
    }

In [12]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [14]:
dataset_samsum_pt['test']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [15]:
dataset_samsum_pt['validation']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
})

In [16]:
#Training the model
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus) #Initialize the model

In [17]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    # Directory where the trained model and checkpoints will be saved
    output_dir='pegasus-samsum',

    # Number of training epochs (complete passes through the dataset)
    num_train_epochs=1,

    # Number of warmup steps for learning rate scheduling
    warmup_steps=500,

    # Batch size for training on each device (GPU/CPU)
    per_device_train_batch_size=1,

    # Batch size for evaluation on each device (GPU/CPU)
    per_device_eval_batch_size=1,

    # Weight decay to apply to the model's parameters to reduce overfitting
    weight_decay=0.01,

    # Number of steps between logging updates (progress reports during training)
    logging_steps=10,

    # Strategy for evaluation: 'steps' means evaluate every `eval_steps` steps
    evaluation_strategy='steps',

    # Number of steps between evaluation runs
    eval_steps=500,

    # Save model checkpoints every `save_steps` steps (here set to a very high value to effectively disable)
    save_steps=10e7,

    # Number of steps to accumulate gradients before performing a backward pass
    gradient_accumulation_steps=1,

    # Disable wandb integration
    report_to="none",

    fp16=True,  # Enable mixed precision training

    gradient_checkpointing=False  # Disable gradient checkpointing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer= Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,
                 data_collator=seq2seq_data_collator, train_dataset=dataset_samsum_pt['train'],
                 eval_dataset=dataset_samsum_pt['validation'])

<ipython-input-18-4206e3b5303b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer= Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,


In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [20]:
torch.cuda.empty_cache()

In [21]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [22]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.988700,1.642061
1000,1.874200,1.549954
1500,1.601200,1.517345
2000,2.050200,1.509550
2500,1.617300,1.479755
3000,1.632100,1.461887
3500,1.710800,1.474794
4000,1.634600,1.441966
4500,2.125300,1.432941
5000,2.759100,1.425312


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=14732, training_loss=1.5815081378703464, metrics={'train_runtime': 7194.6516, 'train_samples_per_second': 2.048, 'train_steps_per_second': 2.048, 'total_flos': 5531718781673472.0, 'train_loss': 1.5815081378703464, 'epoch': 1.0})

In [24]:
# Function to split a list into smaller chunks of a given batch size
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Splits a list into smaller chunks of the specified batch size.

    Args:
        list_of_elements (list): The list to split into chunks.
        batch_size (int): The size of each chunk.

    Yields:
        list: A chunk of the list with length equal to batch_size.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

# Function to evaluate the model on the test dataset and compute a given metric
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text='article',
                                column_summary='highlights'):
    """
    Evaluates a summarization model on a test dataset and calculates the given metric.

    Args:
        dataset (Dataset): The dataset containing articles and reference summaries.
        metric (Metric): The metric to compute (e.g., ROUGE, BLEU).
        model (PreTrainedModel): The trained summarization model.
        tokenizer (PreTrainedTokenizer): The tokenizer for encoding and decoding text.
        batch_size (int): The number of samples to process at once. Default is 16.
        device (torch.device): The device (CPU/GPU) for model computation.
        column_text (str): The column name in the dataset containing the input text (articles).
        column_summary (str): The column name in the dataset containing reference summaries.

    Returns:
        dict: The computed metric scores.
    """
    # Split the input articles into batches of size `batch_size`
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    # Split the target summaries into batches of size `batch_size`
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    # Loop through each batch of articles and their corresponding summaries
    for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

        # Tokenize the batch of articles
        inputs = tokenizer(article_batch,
                           max_length=1024,  # Maximum length of input sequences
                           truncation=True,  # Truncate input sequences longer than max_length
                           padding='max_length',  # Pad input sequences to max_length
                           return_tensors='pt')  # Return PyTorch tensors

        # Generate summaries using the model
        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),  # Move input IDs to the specified device
            attention_mask=inputs["attention_mask"].to(device),  # Move attention mask to device
            length_penalty=0.8,  # Encourage shorter summaries (lower length penalty)
            num_beams=8,  # Use beam search with 8 beams for better summaries
            max_length=128  # Maximum length of generated summaries
        )

        # Decode the generated summaries into text
        decoded_summaries = [
            tokenizer.decode(s,
                             skip_special_tokens=True,  # Remove special tokens (e.g., <s>, </s>)
                             clean_up_tokenization_spaces=True)  # Clean up spaces in decoded text
            for s in summaries
        ]

        # Replace empty strings in decoded summaries with a single space (clean-up step)
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        # Add the predictions and reference summaries to the metric for evaluation
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute the metric scores (e.g., ROUGE scores)
    score = metric.compute()
    return score


In [27]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a378134488f5e10bd1a3b2d2ee96f1514db1d29f242724c76b546f19a9752575
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [28]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=evaluate.load('rouge')

In [31]:
score=calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
)

rouge_dict=dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f'pegasus'])

100%|██████████| 5/5 [00:11<00:00,  2.30s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023044,0.0,0.022947,0.022959


In [32]:
#save the model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [33]:
# save the tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [34]:
#load
tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

In [37]:
#prediction

# Define the generation parameters for the model
gen_kwargs = {
    "length_penalty": 0.2,  # Controls the length of the summary; lower values encourage shorter summaries
    "num_beams": 8,         # Number of beams for beam search; higher value improves quality but is slower
    "max_length": 128       # Maximum number of tokens in the summary
}

# Select a sample dialogue (input text) from the test dataset
sample_text = dataset_samsum["test"][0]["dialogue"]

# Select the reference summary (ground truth summary) for the sample dialogue
reference = dataset_samsum["test"][0]["summary"]

# Create a pipeline for text summarization using the trained model and tokenizer
pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

# Print the original dialogue (input text for summarization)
print("Dialogue: ")
print(sample_text)

# Print the reference summary (what the model is expected to generate)
print("\nReference Summary: ")
print(reference)

# Use the trained model to generate a summary for the sample dialogue
print("\nModel Summary: ")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])  # Pass the sample text and generation parameters


Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue: 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary: 
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary: 
Amanda can't find Betty's number. Larry called Betty's last time they were at the park together. Hannah wants Amanda to text Larry.
